# 4. 推理與評估

模型經過 LoRA 微調後，本筆記本將示範如何使用這個帶有 PEFT adapter 的模型進行推理，並觀察其在特定任務上的表現。

## 步驟 1: 準備模型進行推理

在訓練完成後，`peft` 模型會自動儲存 adapter 的權重。要進行推理，我們需要重新載入基礎模型和 adapter。

**重要提示**: 由於我們在訓練時使用了 4-bit 量化，推理時也必須以相同的量化設定載入基礎模型，否則將無法正確載入 adapter 權重。

我們將從 `./lora-llama2-7b-chat` 目錄中最新的 checkpoint 載入我們訓練好的 LoRA adapter。


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

# --- 重新載入基礎模型與 Tokenizer ---
# (與 02, 03 筆記本中的設定相同)
model_id = "meta-llama/Llama2-7b-chat-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# --- 載入 LoRA Adapter ---
# 找到最新的 checkpoint
output_dir = "./lora-llama2-7b-chat"
latest_checkpoint = max([os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint-")], key=os.path.getmtime)
print(f"從 {latest_checkpoint} 載入 adapter")

# 從 checkpoint 載入 PeftModel
inference_model = PeftModel.from_pretrained(base_model, latest_checkpoint)

print("
模型準備完成，可以開始推理。")


## 步驟 2: 進行文本生成

現在我們可以使用微調後的模型來生成文本了。我們將提供一個提示 (prompt)，並觀察模型的回答是否比原始的 Llama-2-chat 模型更符合我們的微調資料集風格。

`guanaco-llama2-1k` 資料集的風格是問答式的。我們將使用一個類似的問題來測試模型。

推理流程如下：
1.  將提示文本編碼為 `input_ids`。
2.  使用模型的 `generate()` 方法生成文本。
3.  將生成的 `output_ids` 解碼回人類可讀的文本。

`generate()` 方法的參數：
- `max_new_tokens`: 控制生成文本的最大長度。
- `do_sample=True`: 啟用採樣，使生成更多樣。
- `top_k`: Top-K 採樣。
- `num_return_sequences`: 指定生成幾個不同的序列。


In [ ]:
# 準備提示
prompt = "請向我介紹一下台灣的玉山國家公園"
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

# 進行推理
# attention_mask is not strictly necessary for generation with Llama models, 
# but it's good practice to include it.
with torch.no_grad():
    outputs = inference_model.generate(
        input_ids=input_ids, 
        max_new_tokens=256, 
        do_sample=True, 
        top_k=50,
        num_return_sequences=1
    )

# 解碼並打印結果
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("模型生成的回應：")
print(generated_text)


## (可選) 步驟 3: 與基礎模型比較

為了更好地評估 LoRA 微調的效果，您可以載入未經微調的原始 Llama-2 模型，並用相同的提示進行推理，然後比較兩者的輸出差異。

---

推理完成！在最後一個筆記本 `05-merge-and-save.ipynb` 中，我們將學習如何將 LoRA adapter 合併回基礎模型，以便於未來部署。
